In [1]:
import logging
import os
import sched
import sys
import time
import pandas as pd
from datetime import datetime
from models.PyCryptoBot import PyCryptoBot
from models.AppState import AppState
from models.Trading import TechnicalAnalysis
from models.TradingAccount import TradingAccount
from models.helper.LogHelper import Logger

app = PyCryptoBot()
state = AppState()
config = {}
trading_historical_data = {}
account = None
account = TradingAccount(app)
state.last_action = "SELL"
dataframe_dict = []
pd.set_option('display.max_columns', None)

In [23]:
def getAction(now: datetime = datetime.today().strftime('%Y-%m-%d %H:%M:%S'), app: PyCryptoBot = None, price: float = 0,
              df: pd.DataFrame = pd.DataFrame(), df_last: pd.DataFrame = pd.DataFrame(), last_action: str = 'WAIT') -> str:
    ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
    macdgtsignal = bool(df_last['macdgtsignal'].values[0])
    goldencross = bool(df_last['goldencross'].values[0])
    obv_pc = float(df_last['obv_pc'].values[0])
    elder_ray_buy = bool(df_last['eri_buy'].values[0])
    ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
    macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
    ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
    macdltsignal = bool(df_last['macdltsignal'].values[0])
    rri_buy = bool(df_last['rri_buy'].values[0])

    action = '' 
#     print(df_last['open'], df_last['close'], df_last['high'], df_last['low'])
    # criteria for a buy signal
    if ema12gtema26co is True \
            and (macdgtsignal is True or app.disableBuyMACD()) \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26co: {ema12gtema26co}')

        if not app.disableBuyMACD():
            Logger.debug(f'macdgtsignal: {macdgtsignal}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif ema12gtema26 is True \
            and macdgtsignalco is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26: {ema12gtema26}')
        Logger.debug(f'macdgtsignalco: {macdgtsignalco}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif rri_buy is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':
        
        action = 'BUY'

    # criteria for a sell signal
    elif ema12ltema26co is True \
            and (macdltsignal is True or app.disableBuyMACD()) \
            and last_action not in ['', 'SELL']:

        action = 'SELL'

        Logger.debug('*** Sell Signal ***')
        Logger.debug(f'ema12ltema26co: {ema12ltema26co}')
        Logger.debug(f'macdltsignal: {macdltsignal}')
        Logger.debug(f'last_action: {last_action}')

    # anything other than a buy or sell, just wait
    else:
        action = 'WAIT'

    # if disabled, do not buy within 3% of the dataframe close high
    if last_action == 'SELL' and app.disableBuyNearHigh() is True and (price > (df['close'].max() * 0.97)):
        log_text = str(now) + ' | ' + app.getMarket() + ' | ' + \
            app.printGranularity() + ' | Ignoring Buy Signal (price ' + str(price) + ' within 3% of high ' + str(
            df['close'].max()) + ')'
        Logger.warning(log_text)

        action = 'WAIT'

    return action


def getInterval(df: pd.DataFrame = pd.DataFrame(), app: PyCryptoBot = None, iterations: int = 0) -> pd.DataFrame:
    if len(df) == 0:
        return df
    return df.iloc[iterations - 1:iterations]


In [24]:
def find_high_low_interval(df, start, next_x, price):
    df_clipped = df.iloc[start:start+next_x]
    high_pct = round(((df_clipped["high"].max()/price) - 1) * 100, 2)
    low_pct = round(((df_clipped["low"].min()/price) - 1) * 100, 2)
    return (high_pct, low_pct)

In [25]:
def detect_historical_buys(coin_pair):
    state.iterations = 0
    print("calculating ", coin_pair)
    if coin_pair not in trading_historical_data:
        trading_historical_data[coin_pair] = app.getHistoricalData(coin_pair, 3600)
    trading_data = trading_historical_data[coin_pair]
    trading_dataCopy = trading_data.copy()
    ta = TechnicalAnalysis(trading_dataCopy)
    ta.addAll()
    df = ta.getDataFrame()
    while (state.iterations < 300):
        state.iterations = state.iterations + 1
        df_last = getInterval(df, app, state.iterations)
        if len(df_last) > 0:
            now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            price = float(df_last['close'].values[0])
            if price < 0.0001:
                continue
                raise Exception(app.getMarket() + ' is unsuitable for trading, quote price is less than 0.0001!')

            # technical indicators
            ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
            ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
            goldencross = bool(df_last['goldencross'].values[0])
            macdgtsignal = bool(df_last['macdgtsignal'].values[0])
            macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
            ema12ltema26 = bool(df_last['ema12ltema26'].values[0])
            ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
            macdltsignal = bool(df_last['macdltsignal'].values[0])
            macdltsignalco = bool(df_last['macdltsignalco'].values[0])
            obv = float(df_last['obv'].values[0])
            obv_pc = float(df_last['obv_pc'].values[0])
            elder_ray_buy = bool(df_last['eri_buy'].values[0])
            elder_ray_sell = bool(df_last['eri_sell'].values[0])
            rri = df_last['rri'].values[0]
            last_3_rri_buy = df_last['last_3_rri_buy'].values[0]
            last_5_rri_buy = df_last['last_5_rri_buy'].values[0]
            last_3_rri_sell = df_last['last_3_rri_sell'].values[0]
            last_5_rri_sell = df_last['last_5_rri_sell'].values[0]
            rri_buy = df_last['rri_buy'].values[0]
            rri_sell = df_last['rri_sell'].values[0]
            # if simulation interations < 200 set goldencross to true
            if app.isSimulation() and state.iterations < 200:
                goldencross = True

            state.action = getAction(now, app, price, df, df_last, state.last_action)
            if state.action=="BUY":
                one_hr_high, one_hr_low = find_high_low_interval(df, state.iterations, 1, price)
                three_hr_high, three_hr_low = find_high_low_interval(df, state.iterations, 3, price)
                six_hr_high, six_hr_low = find_high_low_interval(df, state.iterations, 6, price)
                nine_hr_high, nine_hr_low = find_high_low_interval(df, state.iterations, 9, price)
                twelve_hr_high, twelve_hr_low = find_high_low_interval(df, state.iterations, 12, price)
                
                three_six_high, three_six_low = find_high_low_interval(df, state.iterations+3, 6, price)
                six_12_high, six_12_low = find_high_low_interval(df, state.iterations+6, 12, price)
                dataframe_dict.append([state.action, app.getGranularity(), price, ema12gtema26, ema12gtema26co,
                                     goldencross, macdgtsignal, macdgtsignalco, obv, obv_pc, elder_ray_buy,
                                     elder_ray_sell, rri, rri_buy, rri_sell, last_3_rri_buy, last_5_rri_buy, last_3_rri_sell, last_5_rri_sell, one_hr_high, one_hr_low, three_hr_high, three_hr_low, 
                                     three_six_high,three_six_low, six_12_high, six_12_low, six_hr_high, six_hr_low,
                                     nine_hr_high, nine_hr_low, twelve_hr_high, twelve_hr_low])
    

In [26]:
%%time
app = PyCryptoBot()
state = AppState()
binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB', 'XLMBNB', 'WABIBNB', 'LTCBNB', 'WAVESBNB', 'ICXBNB',
                          'BLZBNB', 'ZILBNB', 'ONTBNB', 'WANBNB', 'ADABNB', 'ZENBNB', 'EOSBNB', 'THETABNB', 'XRPBNB',
                          'ENJBNB', 'TRXBNB', 'ETCBNB', 'SCBNB', 'MFTBNB', 'VETBNB', 'RVNBNB', 'MITHBNB', 'BTTBNB',
                          'HOTBNB', 'FETBNB', 'XMRBNB', 'ZECBNB', 'IOSTBNB', 'CELRBNB', 'DASHBNB', 'MATICBNB',
                          'ATOMBNB', 'ONEBNB', 'FTMBNB', 'ALGOBNB', 'ANKRBNB', 'WINBNB', 'COSBNB', 'COCOSBNB',
                          'PERLBNB', 'CHZBNB', 'BANDBNB', 'XTZBNB', 'HBARBNB', 'STXBNB', 'KAVABNB', 'ARPABNB', 'BCHBNB',
                          'TROYBNB', 'FTTBNB', 'OGNBNB', 'WRXBNB', 'MBLBNB', 'COTIBNB', 'SOLBNB', 'CTSIBNB', 'CHRBNB',
                          'STMXBNB', 'IQBNB', 'DGBBNB', 'SXPBNB', 'SNXBNB', 'VTHOBNB', 'MKRBNB', 'RUNEBNB', 'FIOBNB',
                          'AVABNB', 'YFIBNB', 'SRMBNB', 'ANTBNB', 'CRVBNB', 'SANDBNB', 'OCEANBNB', 'NMRBNB',
                          'DOTBNB', 'LUNABNB', 'RSRBNB', 'PAXGBNB', 'WNXMBNB', 'SUSHIBNB', 'YFIIBNB', 'KSMBNB',
                          'EGLDBNB', 'DIABNB', 'BELBNB', 'WINGBNB', 'SWRVBNB', 'CREAMBNB', 'UNIBNB', 'AVAXBNB',
                          'BAKEBNB', 'BURGERBNB', 'CAKEBNB', 'SPARTABNB', 'XVSBNB', 'ALPHABNB', 'AAVEBNB', 'NEARBNB',
                          'FILBNB', 'INJBNB', 'CTKBNB', 'KP3RBNB', 'AXSBNB', 'HARDBNB', 'UNFIBNB', 'PROMBNB', 'BIFIBNB',
                          'ICPBNB', 'BTCUSDT', 'ETHUSDT', 'BCCUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT',
                       'XRPUSDT', 'EOSUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT',
                       'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'BCHABCUSDT', 'BCHSVUSDT',
                       'LINKUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT',
                       'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'NANOUSDT',
                       'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT',
                       'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT',
                       'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT',
                       'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT',
                       'BANDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT',
                       'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'MCOUSDT', 'CTXCUSDT', 'BCHUSDT',
                       'TROYUSDT', 'VITEUSDT', 'FTTUSDT', 'OGNUSDT', 'DREPUSDT', 'BULLUSDT', 'BEARUSDT',
                       'ETHBULLUSDT', 'ETHBEARUSDT', 'TCTUSDT', 'WRXUSDT', 'BTSUSDT', 'LSKUSDT', 'BNTUSDT', 'LTOUSDT',
                       'EOSBULLUSDT', 'EOSBEARUSDT', 'XRPBULLUSDT', 'XRPBEARUSDT', 'STRATUSDT', 'AIONUSDT', 'MBLUSDT',
                       'COTIUSDT', 'STPTUSDT', 'WTCUSDT', 'DATAUSDT', 'XZCUSDT',
                       'SOLUSDT', 'CTSIUSDT', 'HIVEUSDT', 'CHRUSDT', 'GXSUSDT', 'ARDRUSDT',
                       'LENDUSDT', 'MDTUSDT', 'STMXUSDT', 'KNCUSDT', 'REPUSDT', 'LRCUSDT', 'PNTUSDT', 'COMPUSDT',
                       'BKRWUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'VTHOUSDT', 'DGBUSDT', 'GBPUSDT', 'SXPUSDT',
                       'MKRUSDT', 'DCRUSDT', 'STORJUSDT', 'MANAUSDT', 'YFIUSDT', 'BALUSDT', 'BLZUSDT', 'IRISUSDT', 'KMDUSDT',
                       'SRMUSDT', 'ANTUSDT', 'CRVUSDT', 'SANDUSDT', 'OCEANUSDT', 'NMRUSDT', 'DOTUSDT',
                       'LUNAUSDT', 'RSRUSDT', 'WNXMUSDT', 'TRBUSDT', 'BZRXUSDT', 'SUSHIUSDT', 'YFIIUSDT',
                       'KSMUSDT', 'EGLDUSDT', 'DIAUSDT', 'RUNEUSDT', 'FIOUSDT', 'UMAUSDT', 'BELUSDT',
                       'WINGUSDT', 'UNIUSDT', 'NBSUSDT', 'OXTUSDT', 'SUNUSDT', 'AVAXUSDT',
                       'HNTUSDT', 'FLMUSDT', 'ORNUSDT', 'UTKUSDT', 'XVSUSDT', 'ALPHAUSDT',
                       'AAVEUSDT', 'NEARUSDT', 'FILUSDT',
                        'INJUSDT', 'AUDIOUSDT', 'CTKUSDT',
                       'AKROUSDT', 'AXSUSDT', 'HARDUSDT', 'DNTUSDT', 'STRAXUSDT', 'UNFIUSDT', 'ROSEUSDT', 'AVAUSDT',
                       'XEMUSDT', 'SKLUSDT', 'SUSDUSDT',
                        'GRTUSDT', 'JUVUSDT', 'PSGUSDT', '1INCHUSDT', 'REEFUSDT', 'OGUSDT',
                       'ATMUSDT', 'ASRUSDT', 'CELOUSDT', 'RIFUSDT', 'BTCSTUSDT', 'TRUUSDT', 'CKBUSDT', 'TWTUSDT',
                       'FIROUSDT', 'LITUSDT', 'SFPUSDT', 'DODOUSDT', 'CAKEUSDT', 'ACMUSDT', 'BADGERUSDT', 'FISUSDT',
                       'OMUSDT', 'PONDUSDT', 'DEGOUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'RAMPUSDT', 'SUPERUSDT',
                       'CFXUSDT', 'EPSUSDT', 'AUTOUSDT', 'TKOUSDT', 'PUNDIXUSDT', 'TLMUSDT', 'BTGUSDT', 'MIRUSDT', 'BARUSDT', 'FORTHUSDT', 'BAKEUSDT', 'BURGERUSDT',
                       'SLPUSDT', 'SHIBUSDT', 'ICPUSDT', 'ARUSDT', 'POLSUSDT', 'MDXUSDT', 'MASKUSDT', 'BTCBUSD', 'XRPBUSD', 'ETHBUSD', 'LTCBUSD', 'LINKBUSD', 'ETCBUSD', 'TRXBUSD',
                       'EOSBUSD', 'XLMBUSD', 'ADABUSD', 'BCHBUSD', 'QTUMBUSD', 'VETBUSD', 'ICXBUSD',
                       'BNTBUSD', 'ATOMBUSD', 'DASHBUSD', 'NEOBUSD', 'WAVESBUSD', 'XTZBUSD', 'BATBUSD', 'ENJBUSD',
                       'NANOBUSD', 'ONTBUSD', 'RVNBUSD', 'ALGOBUSD', 'BTTBUSD', 'TOMOBUSD', 'XMRBUSD', 'ZECBUSD',
                       'DATABUSD', 'SOLBUSD', 'CTSIBUSD', 'HBARBUSD', 'MATICBUSD', 'WRXBUSD', 'ZILBUSD', 'KNCBUSD',
                       'LRCBUSD', 'IQBUSD', 'GBPBUSD', 'DGBBUSD', 'COMPBUSD', 'SXPBUSD', 'SNXBUSD', 'MKRBUSD',
                       'RUNEBUSD', 'MANABUSD', 'DOGEBUSD', 'ZRXBUSD', 'FIOBUSD', 'AVABUSD', 'IOTABUSD',
                       'BALBUSD', 'YFIBUSD', 'SRMBUSD', 'ANTBUSD', 'CRVBUSD', 'SANDBUSD', 'OCEANBUSD',
                       'NMRBUSD', 'DOTBUSD', 'LUNABUSD', 'IDEXBUSD', 'RSRBUSD', 'TRBBUSD', 'BZRXBUSD', 'SUSHIBUSD',
                       'YFIIBUSD', 'KSMBUSD', 'EGLDBUSD', 'DIABUSD', 'BELBUSD', 'SWRVBUSD', 'WINGBUSD', 'CREAMBUSD',
                       'UNIBUSD', 'AVAXBUSD', 'FLMBUSD', 'CAKEBUSD', 'XVSBUSD', 'ALPHABUSD', 'VIDTBUSD', 'AAVEBUSD',
                       'NEARBUSD', 'FILBUSD', 'INJBUSD', 'AERGOBUSD', 'ONEBUSD', 'AUDIOBUSD', 'CTKBUSD', 'KP3RBUSD',
                       'AXSBUSD', 'HARDBUSD', 'DNTBUSD', 'CVPBUSD', 'STRAXBUSD', 'FORBUSD', 'UNFIBUSD', 'FRONTBUSD',
                       'BCHABUSD', 'ROSEBUSD', 'SYSBUSD', 'HEGICBUSD', 'PROMBUSD', 'SKLBUSD', 'COVERBUSD', 'GHSTBUSD',
                       'DFBUSD', 'JUVBUSD', 'PSGBUSD', 'BTCSTBUSD', 'TRUBUSD', 'DEXEBUSD',
                       'CKBBUSD', 'TWTBUSD', 'LITBUSD', 'SFPBUSD', 'FXSBUSD', 'DODOBUSD', 'BAKEBUSD',
                       'UFTBUSD', '1INCHBUSD', 'BANDBUSD', 'GRTBUSD', 'IOSTBUSD', 'OMGBUSD', 'REEFBUSD', 'ACMBUSD',
                       'AUCTIONBUSD', 'PHABUSD', 'TVKBUSD', 'BADGERBUSD', 'FISBUSD', 'OMBUSD', 'PONDBUSD', 'DEGOBUSD',
                       'ALICEBUSD', 'CHZBUSD', 'BIFIBUSD', 'LINABUSD', 'PERPBUSD', 'RAMPBUSD', 'SUPERBUSD', 'CFXBUSD',
                       'XVGBUSD', 'EPSBUSD', 'AUTOBUSD', 'TKOBUSD', 'TLMBUSD', 'BTGBUSD', 'HOTBUSD', 'MIRBUSD',
                       'BARBUSD', 'FORTHBUSD', 'BURGERBUSD', 'SLPBUSD', 'SHIBBUSD', 'ICPBUSD', 'ARBUSD', 'POLSBUSD',
                       'MDXBUSD', 'MASKBUSD']
dataframe_dict = []
for coin_pair in binance_coin_pairs:
    try:
        detect_historical_buys(coin_pair)
    except Exception as e:
        print(e)
buy_dataframe = pd.DataFrame(dataframe_dict,
                         columns=['action', 'Granularity', 'Price', 'Fast EMA gt', 'EMA CO',
                                  'Golden Cross', 'Macdgtsignal', 'Macdgtsignalco', 'Obv', 'Obv_pc',
                                  'Elder_ray_buy', 'elder_ray_sell', 'rri', 'rri_buy', 'rri_sell', 'last_3_rri_buy', 'last_5_rri_buy','last_3_rri_sell', 'last_5_rri_sell', 'one_hr_high', 'one_hr_low', 'three_hr_high',
                                  'three_hr_low', 'three_six_high','three_six_low', 'six_12_high', 'six_12_low',
                                  'six_hr_high', 'six_hr_low', 'nine_hr_high', 'nine_hr_low',
                                  'twelve_hr_high', 'twelve_hr_low'])


calculating  WTCBNB
calculating  BATBNB
calculating  NEOBNB
calculating  IOTABNB
calculating  XLMBNB
calculating  WABIBNB
calculating  LTCBNB
calculating  WAVESBNB
calculating  ICXBNB
calculating  BLZBNB
calculating  ZILBNB
calculating  ONTBNB
calculating  WANBNB
calculating  ADABNB
calculating  ZENBNB
calculating  EOSBNB
calculating  THETABNB
calculating  XRPBNB
calculating  ENJBNB
calculating  TRXBNB
calculating  ETCBNB
calculating  SCBNB
calculating  MFTBNB
calculating  VETBNB
calculating  RVNBNB
calculating  MITHBNB
calculating  BTTBNB
calculating  HOTBNB
calculating  FETBNB
calculating  XMRBNB
calculating  ZECBNB
calculating  IOSTBNB
calculating  CELRBNB
calculating  DASHBNB
calculating  MATICBNB
calculating  ATOMBNB
calculating  ONEBNB
calculating  FTMBNB
calculating  ALGOBNB
calculating  ANKRBNB
calculating  WINBNB
calculating  COSBNB
calculating  COCOSBNB
calculating  PERLBNB
calculating  CHZBNB
calculating  BANDBNB
calculating  XTZBNB
calculating  HBARBNB
calculating  STXBNB
c

calculating  ADABUSD
calculating  BCHBUSD
calculating  QTUMBUSD
calculating  VETBUSD
calculating  ICXBUSD
calculating  BNTBUSD
calculating  ATOMBUSD
calculating  DASHBUSD
calculating  NEOBUSD
calculating  WAVESBUSD
calculating  XTZBUSD
calculating  BATBUSD
calculating  ENJBUSD
calculating  NANOBUSD
calculating  ONTBUSD
calculating  RVNBUSD
calculating  ALGOBUSD
calculating  BTTBUSD
calculating  TOMOBUSD
calculating  XMRBUSD
calculating  ZECBUSD
calculating  DATABUSD
calculating  SOLBUSD
calculating  CTSIBUSD
calculating  HBARBUSD
calculating  MATICBUSD
calculating  WRXBUSD
calculating  ZILBUSD
calculating  KNCBUSD
calculating  LRCBUSD
calculating  IQBUSD
calculating  GBPBUSD
calculating  DGBBUSD
calculating  COMPBUSD
calculating  SXPBUSD
calculating  SNXBUSD
calculating  MKRBUSD
calculating  RUNEBUSD
calculating  MANABUSD
calculating  DOGEBUSD
calculating  ZRXBUSD
calculating  FIOBUSD
calculating  AVABUSD
calculating  IOTABUSD
calculating  BALBUSD
calculating  YFIBUSD
calculating  SRMB

In [27]:
buy_dataframe.describe()

,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,16462.0,16462.000000,1.646200e+04,16462.00,16453.000000,16255.000000,16255.000000,16255.000000,16255.000000,16137.000000,16137.000000,15952.000000,15952.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000,16255.000000
mean,3600.0,321.969533,-4.596800e+07,inf,-0.517571,1.518771,-1.272664,2.476954,-2.253782,3.333227,-3.652688,4.566781,-5.132380,3.556570,-3.334972,4.296552,-4.216252,4.962314,-4.843082
std,0.0,3300.123883,1.020375e+09,NaN,0.607941,2.190994,1.422320,3.269906,2.190901,5.968820,4.328730,9.272029,5.871014,4.528275,3.046367,5.703517,3.753303,7.862992,4.282689
min,3600.0,0.000100,-5.653602e+10,-41264.75,-1.000000,-1.950000,-21.090000,-1.750000,-21.090000,-12.690000,-78.180000,-16.210000,-78.740000,-1.160000,-21.090000,-0.890000,-78.180000,-0.890000,-78.740000
25%,3600.0,0.067342,-1.057307e+05,-8.31,-0.889401,0.480000,-1.780000,0.810000,-3.130000,0.340000,-5.910000,0.390000,-8.582500,1.120000,-4.610000,1.310000,-6.010000,1.480000,-6.830000
50%,3600.0,0.826900,2.338567e+04,-0.64,-0.806931,1.040000,-0.920000,1.670000,-1.660000,2.330000,-2.850000,3.100000,-4.120000,2.430000,-2.440000,2.910000,-3.110000,3.270000,-3.580000
75%,3600.0,7.066000,1.150044e+06,7.79,-0.301587,1.940000,-0.330000,3.130000,-0.720000,4.970000,-0.850000,6.730000,-1.217500,4.520000,-1.170000,5.350000,-1.470000,6.030000,-1.710000
max,3600.0,48005.790000,1.351265e+10,inf,1.000000,140.260000,2.110000,140.260000,1.870000,185.200000,21.390000,397.840000,38.530000,140.260000,1.830000,185.200000,1.830000,397.840000,1.720000


### Observation

**In the first hour,** 

- 50% of the BUY signals give 1.11% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.45% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour


- If invested Rs.100 and for 100 buy signals sellupprpcnt is 0.45, definite chance of making Rs.139 if 75% is succesful
- If invested Rs.100 and for 100 buy signals sellupprpcnt is 1.11, definite chance of making Rs.171 if 50% is succesful

- Filterd by ones that makes < 0.05 in first hour and six_hr_high is also less than 0.00 margin, there is none. Which means, even if it does. not make more than 0.05 margin in first hour, at some point it makes more than 0% profit

**In the three hours,**

- 50% of the BUY signals give 2.0% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.8% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour



In [31]:
rri_buys = buy_dataframe.query("last_3_rri_buy==True")
rri_buys.describe()

,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,12333.0,12333.000000,1.233300e+04,12333.000000,12331.000000,12131.000000,12131.000000,12131.000000,12131.000000,12015.000000,12015.000000,11834.000000,11834.000000,12131.000000,12131.000000,12131.000000,12131.000000,12131.000000,12131.000000
mean,3600.0,285.255188,-4.065250e+07,54.330838,-0.830255,1.495886,-1.202929,2.381701,-2.178466,3.237218,-3.615516,4.566940,-5.091933,3.419580,-3.311502,4.132908,-4.185608,4.830422,-4.826662
std,0.0,3079.606256,9.974522e+08,5098.715291,0.225897,1.699724,1.407974,2.771742,2.172012,5.691762,4.384932,8.831602,5.928732,4.079408,3.103805,5.312085,3.823108,7.276061,4.371856
min,3600.0,0.000100,-5.653602e+10,-15887.230000,-1.000000,-1.750000,-21.090000,-1.750000,-21.090000,-12.690000,-78.180000,-13.130000,-78.740000,-1.160000,-21.090000,-0.890000,-78.180000,-0.820000,-78.740000
25%,3600.0,0.055910,-1.428275e+05,-8.830000,-0.919702,0.520000,-1.660000,0.840000,-3.070000,0.390000,-5.920000,0.480000,-8.757500,1.150000,-4.610000,1.340000,-5.990000,1.510000,-6.830000
50%,3600.0,0.776000,1.008732e+04,-1.250000,-0.843750,1.070000,-0.860000,1.680000,-1.560000,2.300000,-2.730000,3.100000,-4.020000,2.430000,-2.360000,2.870000,-3.030000,3.240000,-3.500000
75%,3600.0,6.586000,8.779759e+05,5.480000,-0.790909,1.960000,-0.280000,3.085000,-0.650000,4.820000,-0.730000,6.750000,-1.060000,4.390000,-1.110000,5.190000,-1.370000,5.905000,-1.620000
max,3600.0,47990.880000,1.062463e+10,549098.210000,1.000000,41.200000,2.110000,58.270000,1.870000,185.200000,21.390000,397.840000,38.530000,113.570000,1.830000,185.200000,1.830000,397.840000,1.720000


In [30]:
non_rri_buys = buy_dataframe.query("rri_buy==False")
non_rri_buys.describe()

,Granularity,Price,Obv,Obv_pc,rri,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low
count,4413.0,4413.000000,4.413000e+03,4413.00,4404.000000,4406.000000,4406.000000,4406.000000,4406.000000,4403.000000,4403.000000,4397.000000,4397.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000
mean,3600.0,419.440781,-5.770324e+07,inf,0.423090,1.577946,-1.473833,2.747553,-2.481850,3.556005,-3.788960,4.504494,-5.287271,3.929360,-3.420683,4.735336,-4.326807,5.296552,-4.919553
std,0.0,3826.353487,1.050780e+09,NaN,0.394315,3.180701,1.443534,4.398970,2.217968,6.743878,4.172667,10.328739,5.750181,5.611048,2.871847,6.732784,3.563497,9.249657,4.044682
min,3600.0,0.000104,-3.678390e+10,-41264.75,-1.000000,-1.950000,-15.540000,-1.630000,-17.570000,-10.250000,-24.980000,-16.210000,-35.820000,-0.890000,-19.480000,-0.890000,-24.980000,-0.890000,-35.820000
25%,3600.0,0.092760,-3.004095e+04,-6.26,0.200000,0.360000,-2.040000,0.680000,-3.300000,0.150000,-5.850000,0.120000,-8.250000,1.000000,-4.640000,1.200000,-6.030000,1.340000,-6.820000
50%,3600.0,1.011000,8.707349e+04,2.58,0.500000,0.910000,-1.110000,1.630000,-1.910000,2.360000,-3.160000,3.010000,-4.550000,2.430000,-2.640000,2.970000,-3.330000,3.310000,-3.805000
75%,3600.0,8.061000,2.176180e+06,20.96,0.720942,1.837500,-0.510000,3.290000,-0.950000,5.270000,-1.260000,6.590000,-1.610000,4.930000,-1.430000,5.895000,-1.830000,6.327500,-2.020000
max,3600.0,48005.790000,1.351265e+10,inf,1.000000,140.260000,1.690000,140.260000,1.690000,107.720000,18.980000,394.220000,21.420000,140.260000,1.690000,140.260000,1.690000,394.220000,1.690000


In [ ]:
loss_buys = buy_dataframe.query("three_hr_high<1.1").query("three_six_high<0.05").query("six_12_high<-5")
loss_buys.describe()

In [ ]:
loss_buys_with_sell= loss_buys[loss_buys['Obv_pc'] < -5]
loss_buys_with_sell.describe()

In [ ]:
df_last['rri']

In [ ]:
trading_historical_data

In [ ]:
df_last['rri']